In [ ]:
import os
os.chdir('/home/megatron/work')

import megatron
import pandas as pd
import numpy as np
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Input, Add
from keras import losses, optimizers
from keras.models import Model
import pickle
import sqlite3

In [1]:
from PIL import Image

In [2]:
from scipy.misc import imsave

In [ ]:
inputs = {'X1': Input((48, 48, 1)),
          'X2': Input((48, 48, 1))}
mid_outputs = {}

for input_name, input_node in inputs.items():
    X = Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(48, 48, 1))(input_node)
    X = Conv2D(64, (3, 3), activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    X = Dropout(0.25)(X)
    X = Flatten()(X)
    X = Dense(128, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2, activation='softmax')(X)
    mid_outputs[input_name] = X
    
output = Add()([mid_outputs['X1'], mid_outputs['X2']])

model1 = Model(list(inputs.values()), output)

model1.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
inputs = {'image_pred': Input((2,)),
          'next_image': Input((48, 48, 1))}

imageX = Conv2D(32, kernel_size=(3, 3),
             activation='relu',
             input_shape=(48, 48, 1))(inputs['next_image'])
imageX = Conv2D(64, (3, 3), activation='relu')(imageX)
imageX = MaxPooling2D(pool_size=(2, 2))(imageX)
imageX = Dropout(0.25)(imageX)
imageX = Flatten()(imageX)
imageX = Dense(128, activation='relu')(imageX)
imageX = Dropout(0.5)(imageX)
imageX = Dense(2, activation='softmax')(imageX)

output = Add()([inputs['image_pred'], imageX])

model2 = Model(list(inputs.values()), output)
model2.compile(loss=losses.categorical_crossentropy,
               optimizer=optimizers.Adadelta(),
               metrics=['accuracy'])

In [ ]:
def data_generator(n_obs):
    while True:
        yield {'image1': np.random.random((n_obs, 48, 48, 3)),
               'image2': np.random.random((n_obs, 48, 48, 3)),
               'image3': np.random.random((n_obs, 48, 48, 3)),
               'label1': np.random.randint(0, 2, n_obs),
               'label2': np.random.randint(0, 2, n_obs)
              }

In [ ]:
generator = True

if generator:
    data = data_generator(32)
else:
    data = {'image1': np.random.random((1000, 48, 48, 3)),
            'image2': np.random.random((1000, 48, 48, 3)),
            'image3': np.random.random((1000, 48, 48, 3)),
            'label1': np.random.randint(0, 2, 1000),
            'label2': np.random.randint(0, 2, 1000)}

bw_layer = megatron.layers.RGBtoGrey(method='luminosity', keep_dim=True)

image1 = megatron.nodes.Input('image1', shape=(48, 48, 3))
bw_image1 = bw_layer(image1, 'bw_image')

image2 = megatron.nodes.Input('image2', shape=(48, 48, 3))
bw_image2 = bw_layer(image2, 'bw_image')

label1_in = megatron.nodes.Input('label1')
label1 = megatron.layers.OneHotRange(max_val=1)(label1_in, 'ohe_label1')

model1_layer = megatron.layers.Keras(model1)
pred1 = model1_layer([bw_image1, bw_image2, label1], 'predictor1')

image3 = megatron.nodes.Input('image3', shape=(48, 48, 3))
bw_image3 = bw_layer(image3, 'bw_image')

label2_in = megatron.nodes.Input('label2')
label2 = megatron.layers.OneHotRange(max_val=1)(label2_in, 'ohe_label2')

model2_layer = megatron.layers.Keras(model2)
pred2 = model2_layer([pred1, bw_image3, label2], 'predictor2')

storage = None if generator else sqlite3.connect('megatron_default')
pipeline = megatron.Pipeline([image1, image2, image3, label1_in, label2_in], pred2,
                          name='testkeras', version=0.1,
                          storage=storage)

In [ ]:
if generator:
    pipeline.fit_generator(data, steps_per_epoch=5, epochs=2)
    predictions = pipeline.transform_generator(data)
else:
    pipeline.fit(data)
    predictions = pipeline.transform(data)

In [ ]:
predictions['predictor2'].shape

In [ ]:
next(predictions)['predictor2'].shape

In [ ]:
pipeline.storage.read(lookup=['0'])['predictor'].shape

In [ ]:
pipeline.save('test')

In [ ]:
megatron.visuals.pipeline_imsave(pipeline, 'img/keras.png')
megatron.visuals.pipeline_imshow(pipeline)